In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
import numpy, pandas, sklearn, cv2, skimage
print("Tout est OK ✅")

Tout est OK ✅


In [4]:
import sys
from pathlib import Path

# Ajout du dossier src/ au PYTHONPATH
ROOT = Path("..")  # car ton notebook est dans notebooks/
sys.path.append(str(ROOT / "src"))


In [5]:
# Imports
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from src.dataio import scan_pairs, load_image, load_mask, read_labels, id_from_path
from src.features import masked_stats_rgb, shape_features, symmetry_features, texture_glcm, edge_density

IMAGES_DIR = Path('data/images')
MASKS_DIR  = Path('data/masks')
LABELS_XLSX = Path('data/labels.xlsx')  # fourni

pairs = scan_pairs(IMAGES_DIR, MASKS_DIR)
rows = []
for _id, img_p, msk_p in tqdm(pairs):
    img = load_image(img_p)
    msk = load_mask(msk_p)
    feats = {'id': _id}
    feats |= masked_stats_rgb(img, msk)
    feats |= shape_features(msk)
    feats |= symmetry_features(msk)
    feats |= texture_glcm(img, msk)
    feats |= edge_density(img, msk)
    rows.append(feats)

dfX = pd.DataFrame(rows).sort_values('id')
dfX.to_csv('outputs/features_train.csv', index=False)

# fusion labels
labels = read_labels(LABELS_XLSX)
data = dfX.merge(labels, on='id', how='left')
data.to_csv('outputs/features_train_labeled.csv', index=False)
data.head()


0it [00:00, ?it/s]


KeyError: 'id'